In [1]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 97 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 25.7 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 297 kB 34.5 MB/s 
     |████████████████████████████████| 46 kB 3.7 MB/s 
     |████████████████████████████████| 45 kB 2.1 MB/s 
     |████████████████████████████████| 6.4 MB 25.2 MB/s 
     |████████████████████████████████| 126 kB 38.4 MB/s 
     |████████████████████████████████| 294 kB 40.3 MB/s 
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 596 kB 11.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 38.9 MB/s 
     |████████████████████████████████| 287 kB 46.8 MB/s 
     |████████████████████████████████| 59 kB 7.4 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none

In [3]:
from google.colab import drive
drive.mount('/gdrive')


Mounted at /gdrive


In [4]:
from tqdm import tqdm
import re
import pyterrier as pt
import pickle
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
import onir_pt
import pandas as pd
from pyterrier_t5 import MonoT5ReRanker

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [5]:
query_variations =pd.read_csv('/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/data/variations_trec2019_labeled.csv')
query_variations["query"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['original_query'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r, re=re: re.sub('[\W_]', ' ',  r['variation'].lower()), axis=1)
query_variations["variation"] = query_variations.apply(lambda r: r['query'] if r['variation'].strip() == "" else r['variation'], axis=1)
query_variations['qid'] =query_variations['q_id'].astype(str)
query_variations.head(5)

,dataset,q_id,method,transformation_type,original_query,variation,valid,follow_category,query,qid
0,msmarco,1037798,WordSwapNeighboringCharacterSwap,mispelling,who is robert gray,who is robret gray,True,True,who is robert gray,1037798
1,msmarco,104861,WordSwapNeighboringCharacterSwap,mispelling,cost of interior concrete flooring,cost of interior concrete floroing,True,True,cost of interior concrete flooring,104861
2,msmarco,1063750,WordSwapNeighboringCharacterSwap,mispelling,why did the us volunterilay enter ww1,why did the su volunterilay enter ww1,True,True,why did the us volunterilay enter ww1,1063750
3,msmarco,1103812,WordSwapNeighboringCharacterSwap,mispelling,who formed the commonwealth of independent states,who formed the commonweatlh of independent states,True,True,who formed the commonwealth of independent states,1103812
4,msmarco,1106007,WordSwapNeighboringCharacterSwap,mispelling,define visceral,define vsiceral,True,True,define visceral,1106007


# Trec dataset

## A1 : importing data

In [6]:

def save_pickle(path,some_obj):
  with open(path, 'wb') as f:
    pickle.dump(some_obj, f)


def open_pickle(path):
  with open(path, 'rb') as handle:
      b = pickle.load(handle)
  return b 

In [7]:
dataset = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
dataset
path_dataset= "/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/pickle/trec/"+"_dataset"

#save_pickle(path_dataset,dataset)
dataset = open_pickle(path_dataset)

In [7]:
dataset.get_qrels()

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] [starting] https://trec.nist.gov/data/deep/2019qrels-pass.txt
[INFO] [finished] https://trec.nist.gov/data/deep/2019qrels-pass.txt: s] [187kB] [265kB/s]


,qid,docno,label,iteration
0,19335,1017759,0,Q0
1,19335,1082489,0,Q0
2,19335,109063,0,Q0
3,19335,1160863,0,Q0
4,19335,1160871,0,Q0
...,...,...,...,...
9255,1133167,8839920,2,Q0
9256,1133167,8839922,2,Q0
9257,1133167,944810,0,Q0
9258,1133167,949411,0,Q0


In [8]:
#train
train_path  = "irds:msmarco-passage/train"
train_ds =  pt.get_dataset(train_path)
train_topics =train_ds.get_topics()
#path_dataset= "/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/pickle/antique/"+"_train_dataset"
#save_pickle(path_dataset,train_ds)

print("#Q train ",len(train_topics))

[INFO] Please confirm you agree to the MSMARCO data usage agreement found at <http://www.msmarco.org/dataset.aspx>
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/c177b2795d5f2dcc524cf00fcd973be1
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz: [4.49s] [18.9MB] [4.20MB/s]


#Q train  808731


In [9]:
#Test  BM25
import os 
pt_index_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/index/TREC/index_trec'

if not os.path.exists(pt_index_path + "/data.properties"):
  # création de l'index. Utilisation de l'itérateur pour parcourir la collection
  # on donne à l'index la fonction pour parcourir l'index avec l'itérateur  get_corpus_iter() 
  # On spécifie les champs à indexer et les meta-données à sauvegarder
  
  indexer = pt.index.IterDictIndexer(pt_index_path)
  indexref = indexer.index(dataset.get_corpus_iter(), fields=('docno', 'text'))
else:
  # dans le cas où l'index existe déjà
  indexref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(indexref)


## A2 : Test pour differents  




### sans test/train

In [10]:
cut_off = 100
metrics=["map", "recip_rank",'P_10', 'ndcg_cut.10']
baseline=0
retrieval_model_name = ["BM25"]
#topics = test_ds.get_topics()
#qrels = test_ds.get_qrels()
#pipeline
"""
retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off >> pt.text.get_text(train_ds, 'text') 
retrieval_model.fit(
                  train_ds.get_topics(),
                  train_ds.get_qrels(),
                  dataset.get_topics(),
                  dataset.get_qrels())

df = pt.Experiment(
    [retrieval_model],
    topics,
    qrels,
    eval_metrics= metrics
)
"""

'\nretrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off >> pt.text.get_text(train_ds, \'text\') \nretrieval_model.fit(\n                  train_ds.get_topics(),\n                  train_ds.get_qrels(),\n                  dataset.get_topics(),\n                  dataset.get_qrels())\n\ndf = pt.Experiment(\n    [retrieval_model],\n    topics,\n    qrels,\n    eval_metrics= metrics\n)\n'

### Generate scores

In [11]:
#table 5

def pair_iter(dataset):
    ds = dataset.irds_ref()
    doc_lookup = ds.docs_store()
    query_lookup = {q.query_id: q for q in ds.queries_iter()}
    while True:
        for docpair in ds.docpairs_iter():            
            text_a = doc_lookup.get(docpair.doc_id_a).text
            text_b = doc_lookup.get(docpair.doc_id_b).text
            text_query = query_lookup[docpair.query_id].text
            yield onir_pt.TrainPair(docpair.query_id, text_query,
                                    docpair.doc_id_a, text_a,
                                    docpair.doc_id_b, text_b)


def generate_original_scores(dataset,index,retrieval_model_name,metrics,cut_off=100,max_iter=400,train_ds=None):
      
      """
      fonction qui retourne le dataframe des scores des original queries

      **dataset : objet dataset 
      **train_ds : si modele neuronal  (objet data)
      **index : index de pt.Index
      **retrieval_model_name : string
      **metrics : liste de metriques
      **cut_off : param du retrieving

      """
      topics = dataset.get_topics()
      qrels = dataset.get_qrels()

      #modeles classiques
      if retrieval_model_name == "BM25":
        retrieval_model = pt.BatchRetrieve(index, wmodel=retrieval_model_name) % cut_off>> pt.text.get_text(dataset, 'text')
      elif retrieval_model_name == "RM3":
        bm_25 = pt.BatchRetrieve(index, wmodel="BM25") 
        retrieval_model = bm_25 >>  pt.rewrite.RM3(index) >> bm_25
      




      #modeles neuronales 
      elif retrieval_model_name=="KNRM":
        knrm = onir_pt.reranker('knrm', 'wordvec_hash', config={'max_train_it':max_iter, 'pre_validate': False})
        retrieval_model =  pt.BatchRetrieve(index, wmodel="BM25", verbose=True) % cut_off >> pt.text.get_text(dataset, 'text')
 
        valid = retrieval_model(dataset.get_topics())
        knrm.fit(va_run=valid, va_qrels=dataset.get_qrels(),tr_pairs=pair_iter(train_ds))
        model_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/modeles/knrm'+retrieval_model_name+' '+str(max_iter)+' '+'TREC'
        knrm.to_checkpoint(model_path)
      


      elif retrieval_model_name == "T5":
        reranker = MonoT5ReRanker()
        retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off >> pt.text.get_text(dataset, 'text') >> reranker
      

      elif retrieval_model_name=="BERT":
       bert = onir_pt.reranker('vanilla_transformer', 'bert', vocab_config={'train': True}, config={'max_train_it':max_iter, 'learning_rate': 1e-5, 'batch_size': 2,  'pre_validate': False, 'patience':50})
       retrieval_model =  pt.BatchRetrieve(index, wmodel="BM25", verbose=True) % cut_off >> pt.text.get_text(dataset, 'text')
       valid = retrieval_model(dataset.get_topics())
       bert.fit(va_run=valid, va_qrels=dataset.get_qrels(),tr_pairs=pair_iter(train_ds))
       model_path = '/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/modeles/bert'+retrieval_model_name+' '+str(max_iter)+' '+'TREC'
       bert.to_checkpoint(model_path)

      #epic or cknrm
      elif 'http' in retrieval_model_name:
        config = {}
        if 'epic' in retrieval_model_name:
          config = {'batch_size': 4}
        reranker = onir_pt.reranker.from_checkpoint(retrieval_model_name, config=config)
        retrieval_model = pt.BatchRetrieve(index, wmodel="BM25") % cut_off>> pt.text.get_text(train_ds, 'text') >> reranker
      else:
        retrieval_model=" "
      
      """
      df = pt.Experiment(
          [retrieval_model],
          topics,
          qrels,

          eval_metrics= metrics
      )
      """
      df = "" 
      return df,retrieval_model


#### Modeles usuels

In [15]:
#BM25
score_original_BM25 =generate_original_scores(dataset,index,"BM25",metrics)
score_original_BM25

[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: s] [4.28kB] [2.71MB/s]
[INFO] [starting] building docstore
docs_iter:  91%|███████████████▍ | 8019075/8841823 [03:10<19.51s, 42175.11doc/s]
[INFO] [error] docs_iter: [03:10] [8019075doc] [42174.40doc/s]
[INFO] [error] building docstore [03:10]


KeyboardInterrupt: ignored

In [ ]:

#RM3
score_original_RM3 =generate_original_scores(dataset,index,"RM3",metrics)
score_original_RM3


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(BR(BM25), QueryExpansion(/cont...",0.404267,0.79374,0.637209,0.515158


#### Modeles neuronnaux

In [ ]:
score_original_KNRM =generate_original_scores(dataset=dataset,train_ds=train_ds,index=index,retrieval_model_name="KNRM",metrics=metrics,max_iter=50)
score_original_KNRM


[2022-04-26 14:30:24,545][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-26 14:30:36,488][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [11.94s]


In [ ]:
dataset.get_corpus_iter()


msmarco-passage/trec-dl-2019/judged documents:   0%|          | 0/8841823 s<?, ?it/s]

#### CKRNM (link)

In [ ]:
score_original_CKNRM =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz",metrics=metrics,max_iter=100)
score_original_CKNRM

[2022-04-21 15:40:03,419][onir_pt][INFO] using cached checkpoint: /root/data/onir/model_checkpoints/52177bd86fe00fbf6a41fbb843b981b1
[2022-04-21 15:40:03,438][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-21 15:40:15,079][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [11.64s]
[2022-04-21 15:40:31,191][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-21 15:40:31,540][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2022-04-21 15:41:00,657][onir_pt][DEBUG] [finished] batches: [29.11s] [526it] [18.07it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.302319,0.810465,0.625581,0.493117


#### EPIC (link)

In [ ]:
score_original_EPIC =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz",metrics=metrics,max_iter=100)
score_original_EPIC

[2022-04-21 15:42:41,473][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz; consider adding expected_md5="c184a924debf63b5115e34a3b92377e0" to ensure data integrity.
[2022-04-21 15:42:41,489][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz [01:40] [495M] [64.4MB/s]


100%|██████████| 231508/231508 s<0ms, 2609062.88B/s]
100%|██████████| 433/433 s<0ms, 301432.97B/s]
100%|██████████| 440473133/440473133 [13.47s<0ms, 32711520.12B/s] 


[2022-04-21 15:45:10,007][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-21 15:45:10,259][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2022-04-21 15:47:13,944][onir_pt][DEBUG] [finished] batches: [02:04] [1052it] [ 8.51it/s]


,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.345216,0.94186,0.709302,0.623192


#### T5

In [ ]:
score_original_T5 =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="T5",metrics=metrics,max_iter=100)
score_original_T5


Downloading:   0%|          | 0.00/773k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k s<?, ?B/s]

Downloading:   0%|          | 0.00/850M s<?, ?B/s]

monoT5:   0%|          | 0/1052 s<?, ?batches/s]

,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(Compose(RankCutoff(BR(BM25), 100), pt....",0.367611,0.959302,0.793023,0.699547


#### BERT

In [ ]:
score_original_BERT=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="BERT",metrics=metrics,max_iter=5)
score_original_BERT

[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz
[INFO] [finished] https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz: s] [4.28kB] [3.06MB/s]


config file not found: config


100%|██████████| 231508/231508 s<0ms, 2752520.55B/s]
100%|██████████| 433/433 s<0ms, 330146.09B/s]
100%|██████████| 440473133/440473133 [11.43s<0ms, 38533660.12B/s] 


BR(BM25):   0%|          | 0/43 s<?, ?q/s]

[INFO] [starting] building docstore
docs_iter: 100%|████████████████████| 8841823/8841823 [03:31<0ms, 41904.45doc/s]
[INFO] [finished] docs_iter: [03:31] [8841823doc] [41904.17doc/s]
[INFO] [finished] building docstore [03:31]


...

[2022-04-26 13:51:38,200][onir_pt][DEBUG] [finished] train pairs: [03:13] [1024it] [ 5.32it/s]
[2022-04-26 13:51:38,203][onir_pt][DEBUG] [finished] training [03:13]
[2022-04-26 13:51:38,203][onir_pt][INFO] training   it=4 loss=0.0541
[2022-04-26 13:51:38,204][onir_pt][DEBUG] [starting] validation
[2022-04-26 13:51:38,204][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-26 13:51:38,227][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/2103 s<?, ?it/s]

[2022-04-26 13:53:15,091][onir_pt][DEBUG] [finished] batches: [01:37] [2103it] [21.71it/s]
[2022-04-26 13:53:15,152][onir_pt][DEBUG] [finished] validation [01:37]
[2022-04-26 13:53:16,986][onir_pt][INFO] validation it=4 map=0.3503 ndcg=0.4966 P_10=0.7279 <--


BR(BM25):   0%|          | 0/43 s<?, ?q/s]

,name,map,recip_rank,P_10,ndcg_cut.10
0,"Compose(RankCutoff(BR(BM25), 100), pt.apply.??())",0.290719,0.794961,0.597674,0.479592


In [ ]:

#pipeline =pt.BatchRetrieve(index, wmodel="BM25")%cut_off>>bert

# Variations et scores

In [12]:

def get_variation_score(dataset,retrieval_model_name,retrieval_model,query_variations):
  variation_methods = []
  res_per_variation = []
  print()
  for method in query_variations['method'].unique():
    query_variation = query_variations[query_variations['method'] == method]
    print(method)
    query_variation['query'] = query_variation['variation']
    res_per_variation.append(retrieval_model.transform(query_variation[['qid', 'query']]))
    variation_methods.append("{}+QueriesFrom{}".format(retrieval_model_name, method))

  df = pt.Experiment(
        [retrieval_model] + res_per_variation,
        #requetes
        query_variations[['query','qid']].drop_duplicates(),
        #jugement de pertinence
        dataset.get_qrels(),
        metrics,
        baseline=0,
        names = [retrieval_model_name]+variation_methods
        )
  path = "/gdrive/MyDrive/SORBONNE/S2/rital/RI/projet/pickle/trec/variations/"+"_"+retrieval_model_name+"_"+"trec"
  save_pickle(path,df)
  df.head(10)
    
  return df





In [27]:
#BM25
score_original_BM25, retrieval_model =generate_original_scores(dataset,index,"BM25",metrics)
scores_variation_BM25 = get_variation_score(dataset,"BM25",retrieval_model,query_variations)


WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


WordSwapQWERTY
WordSwapRandomCharacterSubstitution
naturality_by_removing_stop_words
summarization_with_t5-base_from_description_to_title
WordInnerSwapRandom
back_translation_pivot_language_de
ramsrigouthamg/t5_paraphraser
WordSwapWordNet
WordSwapEmbedding


In [28]:
#RM3
score_original_RM3,retrieval_model =generate_original_scores(dataset,index,"RM3",metrics)
scores_variation_RM3=get_variation_score(dataset,"RM3",retrieval_model,query_variations)



WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


WordSwapQWERTY
WordSwapRandomCharacterSubstitution
naturality_by_removing_stop_words
summarization_with_t5-base_from_description_to_title
WordInnerSwapRandom
back_translation_pivot_language_de
ramsrigouthamg/t5_paraphraser
WordSwapWordNet
WordSwapEmbedding


In [29]:
#CKNRM
score_original_CKNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz",metrics=metrics,max_iter=100)
scores_variation_CKNRM=get_variation_score(dataset,"CKNRM",retrieval_model,query_variations)


config file not found: config


[2022-04-30 11:37:23,263][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz; consider adding expected_md5="6e57b8d728151debcb23479c8a41f4b0" to ensure data integrity.
[2022-04-30 11:37:23,271][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.convknrm.seed42.tar.gz s] [860k] [7.40MB/s]
[2022-04-30 11:37:23,315][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2022-04-30 11:38:01,627][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2022-04-30 11:38:01,639][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [37.27s] [682M] [10.7MB/s]
[2022-04-30 11:38:01,646][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [38.33s]
[2022-04-30 11:38:01,646][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2022-04-30 11:38:20,834][WordvecHashVocab][DEBUG] [finished] extracting vecs [19.19s]
[2022-04-30 11:38:20,836][WordvecHashVocab][DEBUG] [starting] loading vecs into memory
[2022-04-30 11:40:45,311][WordvecHashVocab][DEBUG] [finished] loading vecs into memory [02:24]
[2022-04-30 11:40:45,448][WordvecHashVocab][DEBUG] [starting] writ

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


[2022-04-30 11:41:03,250][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:41:17,083][onir_pt][DEBUG] [starting] batches


...

[2022-04-30 11:45:27,906][onir_pt][DEBUG] [finished] batches: [24.93s] [525it] [21.06it/s]
WordSwapEmbedding
[2022-04-30 11:45:31,624][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:45:31,626][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2022-04-30 11:45:56,645][onir_pt][DEBUG] [finished] batches: [25.02s] [526it] [21.02it/s]
[2022-04-30 11:46:00,306][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:46:00,308][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/526 s<?, ?it/s]

[2022-04-30 11:46:25,361][onir_pt][DEBUG] [finished] batches: [25.05s] [526it] [21.00it/s]


In [30]:
#EPIC
score_original_EPIC,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz",metrics=metrics,max_iter=100)
scores_variation_EPIC=get_variation_score(dataset,"EPIC",retrieval_model,query_variations)

[2022-04-30 11:46:45,897][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz; consider adding expected_md5="c184a924debf63b5115e34a3b92377e0" to ensure data integrity.
[2022-04-30 11:46:45,904][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.epic.seed42.tar.gz [19.67s] [495M] [26.2MB/s]


100%|██████████| 231508/231508 s<0ms, 319914.88B/s] 
100%|██████████| 433/433 s<0ms, 114021.45B/s]
100%|██████████| 440473133/440473133 [32.85s<0ms, 13407935.48B/s] 



WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


[2022-04-30 11:49:29,956][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 11:49:30,179][onir_pt][DEBUG] [starting] batches


...

[2022-04-30 12:08:08,512][onir_pt][DEBUG] [finished] batches: [02:03] [1050it] [ 8.51it/s]
WordSwapEmbedding
[2022-04-30 12:08:12,389][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 12:08:12,394][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2022-04-30 12:10:16,526][onir_pt][DEBUG] [finished] batches: [02:04] [1052it] [ 8.47it/s]
[2022-04-30 12:10:20,121][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 12:10:20,129][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/1052 s<?, ?it/s]

[2022-04-30 12:12:23,375][onir_pt][DEBUG] [finished] batches: [02:03] [1052it] [ 8.54it/s]


In [31]:
#T5
score_original_T5,retrieval_model =generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="T5",metrics=metrics,max_iter=10)
scores_variation_T5=get_variation_score(dataset,"T5",retrieval_model,query_variations)

Downloading:   0%|          | 0.00/773k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k s<?, ?B/s]

Downloading:   0%|          | 0.00/1.80k s<?, ?B/s]

Downloading:   0%|          | 0.00/850M s<?, ?B/s]


WordSwapNeighboringCharacterSwap


...

WordSwapWordNet


monoT5:   0%|          | 0/1050 s<?, ?batches/s]

WordSwapEmbedding


monoT5:   0%|          | 0/1052 s<?, ?batches/s]

monoT5:   0%|          | 0/1052 s<?, ?batches/s]

In [14]:
#KNRM
score_original_KNRM,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="https://macavaney.us/pt_models/msmarco.knrm.seed42.tar.gz",metrics=metrics,max_iter=5)
scores_variation_KNRM=get_variation_score(dataset,"KNRM",retrieval_model,query_variations)

[2022-04-30 13:16:05,408][onir.util.download][WARNING] no hash provided for https://macavaney.us/pt_models/msmarco.knrm.seed42.tar.gz; consider adding expected_md5="169d1f84eb88c0d3e1f45d12e2c3ca33" to ensure data integrity.
[2022-04-30 13:16:05,422][onir.util.download][DEBUG] downloaded https://macavaney.us/pt_models/msmarco.knrm.seed42.tar.gz s] [1.39k] [89.8kB/s]
[2022-04-30 13:16:05,454][WordvecHashVocab][DEBUG] [starting] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p
[2022-04-30 13:16:18,560][WordvecHashVocab][DEBUG] [finished] reading cached at /root/data/onir/vocab/wordvec_hash/fasttext-wiki-news-300d-1M.p [13.11s]

WordSwapNeighboringCharacterSwap


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
[INFO] [starting] building docstore
[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/31644046b18952c1386cd4564ba2ae69
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
docs_iter:   0%|                                   | 0/8841823 s<?, ?doc/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz: 0.0%| 0.00/1.06G s<?, ?B/s]
https://msmarco.blob.core.windows.net/msmarcoranking/collectiona

[2022-04-30 13:39:32,997][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:39:44,022][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/255 s<?, ?it/s]

[2022-04-30 13:39:45,220][onir_pt][DEBUG] [finished] batches: [1.19s] [255it] [214.65it/s]
WordSwapQWERTY
[2022-04-30 13:39:53,516][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:39:53,518][onir_pt][DEBUG] [starting] batches


...

[2022-04-30 13:40:30,086][onir_pt][DEBUG] [finished] batches: s] [263it] [283.51it/s]
WordSwapEmbedding
[2022-04-30 13:40:34,200][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:40:34,202][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/263 s<?, ?it/s]

[2022-04-30 13:40:35,141][onir_pt][DEBUG] [finished] batches: s] [263it] [280.30it/s]
[2022-04-30 13:40:38,784][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:40:38,785][onir_pt][DEBUG] [starting] batches


batches:   0%|          | 0/263 s<?, ?it/s]

[2022-04-30 13:40:39,728][onir_pt][DEBUG] [finished] batches: s] [263it] [279.37it/s]


In [15]:
#BERT
score_original_BERT,retrieval_model=generate_original_scores(dataset,train_ds=train_ds,index=index,retrieval_model_name="BERT",metrics=metrics,max_iter=3)
scores_variation_BERT=get_variation_score(dataset,"BERT",retrieval_model,query_variations)

100%|██████████| 231508/231508 s<0ms, 318795.66B/s] 
100%|██████████| 433/433 s<0ms, 71492.88B/s]
100%|██████████| 440473133/440473133 [35.83s<0ms, 12293937.69B/s]  


BR(BM25):   0%|          | 0/43 s<?, ?q/s]

[2022-04-30 13:41:36,104][onir_pt][DEBUG] using GPU (deterministic)
[2022-04-30 13:41:36,349][onir_pt][DEBUG] [starting] training
[2022-04-30 13:41:36,350][onir_pt][DEBUG] [starting] train pairs


train pairs:   0%|          | 0/1024 s<?, ?it/s]

[INFO] If you have a local copy of https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/215a5204288820672f5e9451d9e202c5
[INFO] [starting] https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz

https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz: 0.0%| 0.00/2.63G s<?, ?B/s]
https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz: 0.0%| 16.4k/2.63G s<5:56:52, 123kB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz: 0.0%| 49.2k/2.63G s<4:12:14, 174kB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz: 0.0%| 81.9k/2.63G s<3:50:33, 190kB/s]
https://msmarco.blob.core.windows.net/msmarcoranking/qidpidtriples.train.full.tsv.gz: 0.0%| 131k/2.63G s<3:13:29, 227kB/s] 
https://msmarco.blob.core.windows.net/msmarcoran

...

WordSwapWordNet


BR(BM25):   0%|          | 0/42 s<?, ?q/s]

WordSwapEmbedding


BR(BM25):   0%|          | 0/43 s<?, ?q/s]

BR(BM25):   0%|          | 0/43 s<?, ?q/s]

In [16]:
scores_variation_BERT

,name,map,recip_rank,P_10,ndcg_cut.10,map +,map -,map p-value,recip_rank +,recip_rank -,recip_rank p-value,P_10 +,P_10 -,P_10 p-value,ndcg_cut.10 +,ndcg_cut.10 -,ndcg_cut.10 p-value
0,BERT,0.290719,0.794961,0.597674,0.479592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BERT+QueriesFromWordSwapNeighboringCharacterSwap,0.146147,0.457210,0.346512,0.273835,9.0,32.0,0.000057,3.0,24.0,1.089571e-05,6.0,25.0,0.000068,8.0,29.0,0.000123
2,BERT+QueriesFromWordSwapQWERTY,0.139308,0.415726,0.327907,0.243738,7.0,32.0,0.000084,0.0,26.0,1.531041e-07,5.0,28.0,0.000012,5.0,33.0,0.000006
3,BERT+QueriesFromWordSwapRandomCharacterSubstit...,0.138804,0.413398,0.311628,0.231450,9.0,31.0,0.000085,1.0,24.0,2.872312e-06,3.0,29.0,0.000006,7.0,31.0,0.000009
4,BERT+QueriesFromnaturality_by_removing_stop_words,0.293254,0.791860,0.597674,0.477805,3.0,1.0,0.251729,0.0,1.0,3.230373e-01,0.0,0.0,NaN,1.0,2.0,0.551436
5,BERT+QueriesFromsummarization_with_t5-base_fro...,0.259177,0.705635,0.553488,0.421671,11.0,19.0,0.138954,2.0,9.0,1.750032e-02,5.0,13.0,0.138481,9.0,19.0,0.052063
6,BERT+QueriesFromWordInnerSwapRandom,0.290719,0.794961,0.597674,0.479592,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,NaN
7,BERT+QueriesFromback_translation_pivot_languag...,0.219355,0.665676,0.490698,0.396489,4.0,15.0,0.008038,1.0,11.0,1.083964e-02,4.0,13.0,0.016124,4.0,14.0,0.023552
8,BERT+QueriesFromramsrigouthamg/t5_paraphraser,0.290892,0.775028,0.600000,0.472263,2.0,6.0,0.894625,0.0,1.0,3.230373e-01,2.0,2.0,0.710163,2.0,5.0,0.465643
9,BERT+QueriesFromWordSwapWordNet,0.188293,0.634308,0.418605,0.348817,4.0,28.0,0.000545,4.0,15.0,1.115671e-02,2.0,22.0,0.000293,3.0,27.0,0.000721
